# quake 3 server connector



## discord api




### channel message poller



#### the code

poll discord channel?

discord api?



In [ ]:
var WebSocket = require('ws')
var fs = require('fs')
var path = require('path')
var {request} = require('gaxios')
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var credentials
var tokenPath
if(fs.existsSync('./discord-bot.json')) {
    tokenPath = path.resolve('./discord-bot.txt')
} else {
    tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}

async function gateway() {
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `https://discord.com/api/v6/gateway/bot`
    })
    var ws = new WebSocket(result.data.url)
    ws.on('open', () => {
        console.log('Connecting to Discord')
    })
    var heartbeat
    var identified = false
    ws.on('message', (message) => {
        var msgBuff = new Buffer.from(message)
        var gateway = JSON.parse(msgBuff.toString('utf-8'))
        if(gateway.op == 10) {
            heartbeat = setInterval(() => {
                ws.send(JSON.stringify({
                    op: 11
                }))
            }, gateway.d.heartbeat_interval)
            ws.send(JSON.stringify({
                op: 2,
                d: {
                    token: token,
                    properties: {
                        "$os": "linux",
                        "$browser": "jupyter",
                        "$device": "quake3"
                    },
                    intents: 1 << 9 | 1 | 1 << 10
                }
            }))
            return
        } else if (gateway.op === 0) {
            identified = true
            return
        }
        
        console.log(gateway)
    })
    var timer
    await new Promise(resolve => {
        timer = setInterval(() => {
            if(identified) resolve()
        }, 20)
    });
    clearInterval(timer)
}

async function userGuilds(userId = '@me') {
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `https://discord.com/api/v6/users/${userId}/guilds`
    })
    return result.data
}

async function guildChannels(guildId = '393252386426191873') {
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `https://discord.com/api/v6/guilds/${guildId}/channels`
    })
    return result.data
}

async function channelMessages(channelId = '393252386426191875') {
    var params = {
        after: (BigInt(Date.now() - 1420070400000 - 1000 * 65) << 22n).toString()
    };
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `https://discord.com/api/v6/channels/${channelId}/messages`,
        params
    })
    return result.data
}

async function triggerTyping(channelId = '393252386426191875') {
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'POST',
        url: `https://discord.com/api/v6/channels/${channelId}/typing`
    })
    return result.data
}

async function createMessage(message, nonce = false, channelId = '393252386426191875') {
    var params = {
        content: message
    }
    if(nonce) params['nonce'] = nonce
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `https://discord.com/api/v6/channels/${channelId}/messages`,
        data: JSON.stringify(params)
    })
    return result.data
}

module.exports = {
    gateway,
    userGuilds,
    guildChannels,
    channelMessages,
    triggerTyping,
    createMessage
}


#### authorize dicsord oauth


In [ ]:

/*
async function authorizeDiscord() {
    var {client_id, client_secret} = JSON.parse(fs.readFileSync(credentials).toString('utf-8'))
    if(fs.existsSync(tokenPath))
        var {access_token, expires} = JSON.parse(fs.readFileSync(tokenPath).toString('utf-8'))
    if(access_token && Date.now() < expires) return access_token
    var data = `grant_type=client_credentials&scope=identify%20connections%20bot%20guilds%20messages.read%20rpc`
    var result = await request({
        headers: {
            'Content-Type': 'application/x-www-form-urlencoded',
            Authorization: 'Basic ' + Buffer.from(`${client_id}:${client_secret}`).toString('base64')
        },
        method: 'POST',
        url: `https://discord.com/api/v6/oauth2/token`,
        data
    })
    fs.writeFileSync(tokenPath, JSON.stringify(Object.assign({}, result.data, {
        expires: (result.data.expires_in * 1000) + Date.now()}), null, 2))
    return result.data.access_token
}
*/


### respond to channel commands



#### the code

discord q3 bot?

discord commands?


In [ ]:
var importer = require('../Core')
var discordApi = importer.import('discord api')
var serverApi = importer.import('quake 3 server commands')

async function readAllCommands() {
    // matching format  @megamind  challenge freon dm17 , :thumbsup:   :thumbsdown: .
    var messages = []
    var responses = []
    var channels = []
    var commands = []
    var launches = []
    var guilds = await discordApi.userGuilds()
    for(var i = 0; i < guilds.length; i++) {
        channels.push.apply(channels, await discordApi.guildChannels(guilds[i].id))
    }
    
    for(var i = 0; i < channels.length; i++) {
        messages.push.apply(messages, await discordApi.channelMessages(channels[i].id))
    }
    
    // find commands in channel history
    for(var j = 0; j < messages.length; j++) {
        if(messages[j].content.match(/chall?enge/ig) && messages[j].mentions.length > 0) {
            commands.push(messages[j])
            if((messages[j].reactions || [])
                .filter(a => a.emoji.name.match(/:thumbsup:/ig)).length > 0) {
                launches.push(messages[j])
            }
        }
        if((messages[j].nonce || '').match(/^BOT/ig)) {
            responses.push(messages[j])
            if((messages[j].reactions || [])
                .filter(a => a.emoji.name.match(/:thumbsup:/ig)).length > 0) {
                var l = messages.filter(m => m.id == messages[j].nonce)[0]
                if(l) launches.push(l)
            }
        }
    }
    
    // exclude commands that already got a response
    return commands
        .filter(c => responses.filter(r => 'BOT'+c.id == r.nonce).length === 0)
        .concat(launches)
        .filter(c => responses.filter(r => 'BOT'+c.id+'L' == r.nonce).length === 0)
}

async function respondCommand() {
    await discordApi.gateway()
    var commands = await readAllCommands()
    for(var i = 0; i < commands.length; i++) {
        var message = 'I read you,'
        var instruction = ''
        var launch = ''
        var launching = (commands[i].reactions || [])
            .filter(a => a.emoji.name.match(/:thumbsup:/ig)).length > 0
        var map = ''
        var options = (/challenge\s?([^:@\s]*?)\s?([^:@\s]*?)/ig).exec(commands[i].content)
        if(options) {
            launch = options[1]
            map = options[2]
        } else {
            message = 'Got it,'
        }
        if(map.length = 0) {
            map = 'q3dm17'
        }
        if(launch.length == 0) {
            instruction += ' assuming baseq3 on map ' + map
        } else if(launching) {
            message = 'Let\'s play,'
            instruction += ' ' + launch + ' on map ' + map
        }
        if(!commands[i].content.match(/:thumbsup:/ig)) {
            message = 'Waiting for reaction,'
            instruction += ' react with :thumbsup: to launch'
        }
        if(launching) {
            message = 'Launching,'
            await discordApi.createMessage(message + instruction, 'BOT'+commands[i].id+'L', commands[i].channel_id)
            await discordApi.triggerTyping(commands[i].channel_id)
            var masters = await serverApi.listMasters(void 0, void 0, false)
            serverApi.sendRcon(masters[0].ip, masters[0].port, 'map ' + map)
        } else if (instruction.length > 0) {
            await discordApi.createMessage(message + instruction, 'BOT'+commands[i].id, commands[i].channel_id)
        }
    }
}

module.exports = respondCommand


## quake 3 commands



### quake 3 server commands



#### the code

quake 3 server commands?


In [3]:
var path = require('path')
var fs = require('fs')
var zlib = require('zlib')
var dgram = require('dgram')
var udpClient = dgram.createSocket('udp4')
var importer = require('../Core')
var {
    getServersResponse, statusResponse, infoResponse
} = importer.import('quake 3 server responses')
udpClient.on('message', updateInfo)

var MAX_TIMEOUT = 10000

var masters = []
function mergeMaster(master) {
    var found = false
    masters.forEach((ma, i) => {
        if(ma['ip'] == master['ip'] && ma['port'] == master['port']) {
            found = true
            Object.assign(masters[i], master)
            return false
        }
    })
    if(!found)
        masters.push(master)
}

async function updateInfo(m, rinfo) {
    if(m[0] == 255 && m[1] == 255 && m[2] == 255 && m[3] == 255)
        m = m.slice(4, m.length)
    if(m.slice(0, 'getserversResponse'.length).toString('utf-8') == 'getserversResponse') {
        var mas = getServersResponse(m)
        mas.forEach(master => {
            mergeMaster(master)
            getStatus(master.ip, master.port)
        })
    } else if (m.slice(0, 'statusResponse'.length).toString('utf-8') == 'statusResponse') {
        var info = statusResponse(m)
        mergeMaster(Object.assign(info, {
            ip: rinfo.address,
            port: rinfo.port
        }))
    } else if (m.slice(0, 'infoResponse'.length).toString('utf-8') == 'infoResponse') {
        var info = infoResponse(m)
        mergeMaster(Object.assign(info, {
            ip: rinfo.address,
            port: rinfo.port
        }))
    }
}

async function sendRcon(address, port, command, password = 'password123!') {
    var msgBuff = new Buffer.from(`\xFF\xFF\xFF\xFFrcon "${password}" ${command}`.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, address)
}

async function getStatus(address, port) {
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetstatus'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, address)
}

async function getInfo(address, port) {
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetinfo xxx'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, address)
}

async function listMasters(master = '192.168.0.4', port = 27950, wait = true) {
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetservers 68 empty'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, master)
    if(wait) {
        await new Promise(resolve => setTimeout(resolve, MAX_TIMEOUT))
    } else {
        var timeout = 0
        var timer
        await new Promise(resolve => {
            timer = setInterval(() => {
                if(masters.filter(s => s.gamename).length > 0 || timeout == MAX_TIMEOUT / 20) {
                    resolve()
                } else {
                    timeout += 20
                }
            }, 20)
        })
        clearInterval(timer)
    }
    udpClient.close()
    return masters
}

module.exports = {
    listMasters,
    getInfo,
    getStatus,
    sendRcon,
}


SyntaxError: Unexpected token )

### qauke 3 server responses



#### the code

quake 3 server responses?



In [ ]:

function getServersResponse(m) {
    var masters = []
    m = m.slice('getserversResponse'.length)
    for(var i = 0; i < m.length / 7; i++) {
        var ip = i*7+1
        if(m[ip-1] !== '\\'.charCodeAt(0)) continue
        if(m.slice(ip, ip+3) == 'EOT') continue
        var master = {
            ip: m[ip] + '.' + m[ip+1] + '.' + m[ip+2] + '.' + m[ip+3],
            port: (m[ip+4] << 8) + m[ip+5],
        }
        masters.push(master)
    }
    return masters
}

function statusResponse(m) {
    m = m.slice('statusResponse'.length)
    var config = m.toString('utf-8')
        .trim().split(/\n/ig)[0]
        .trim().split(/\\/ig).slice(1)
    return config.reduce((obj, c, i, arr) => {
        if(i & 1) obj[arr[i-1]] = c
        return obj
    }, {})
}

function infoResponse(m) {
    m = m.slice('infoResponse'.length)
    var config = m.toString('utf-8')
        .trim().split(/\n/ig)[0]
        .trim().split(/\\/ig).slice(1)
    return config.reduce((obj, c, i, arr) => {
        if(i & 1) obj[arr[i-1]] = c
        return obj
    }, {})
}

module.exports = {
    getServersResponse,
    statusResponse,
    infoResponse,
}


### Huffman decoder



#### the code



In [ ]:
var Huffman = require('/Users/briancullinan/planet_quake/code/xquakejs/lib/huffman.js')
var MAX_STRING_CHARS = 1024
var buffer
Huffman.onRuntimeInitialized = () => {
    Huffman['_MSG_initHuffman']()
    buffer = Huffman.allocate(new Int8Array(MAX_STRING_CHARS), 'i8', 0)
}


function readBits(m, offset, bits = 8) {
    var value
    var nbits = bits & 7
    var sym = Huffman.allocate(new Int32Array(1), 'i32', 1)
    var bitIndex = offset
    m.forEach((c,i) => Huffman.HEAP8[buffer+i] = c)
    if ( nbits )
    {
        for ( i = 0; i < nbits; i++ ) {
            value |= Huffman._HuffmanGetBit( buffer, bitIndex ) << i
            bitIndex++
        }
        bits -= nbits
    }
    if ( bits )
    {
        for ( i = 0; i < bits; i += 8 )
        {
            bitIndex += Huffman._HuffmanGetSymbol( sym, buffer, bitIndex )
            value |= ( Huffman.getValue(sym) << (i+nbits) )
        }
    }
    return value
}


### find available server



#### the code

